In [1]:
#export
from k1lib.callbacks import Callback, Callbacks

In [2]:
#export
class ParamFinder(Callback):
    """Automatically finds out the right value for
    a specific parameter"""
    def __init__(self, param:str="lr"):
        super().__init__()
        self.potentialValues = 10**_np.linspace(-6, 2, 50)
        self.idx = 0 # values[idx] will be the parameter's value
        self.bestLoss = 1e9
        self.param = param
        self.losses = []
    def startBatch(self):
        if self.idx >= len(self.potentialValues): raise _k1lib.CancelRunException # ran through all possible configs
        for paramGroup in self.opt.param_groups:
            paramGroup[self.param] = self.potentialValues[self.idx]
        self.idx += 1
    def endLoss(self):
        loss = self.loss/self.xb.shape[0] # xb.shape[0] is the batch size
        if loss < self.bestLoss: self.bestLoss = loss
        if loss > self.bestLoss * 10: raise _k1lib.CancelRunException # loss increases significantly
        self.losses.append(loss)
    def run(self):
        self.learner.run(10)
        self.plot()
        return self
    def plot(self):
        if len(self.losses) == 0: print("Has not run param finder yet.")
        else:
            _plt.plot(self.potentialValues[:len(self.losses)], self.losses)
            _plt.xscale("log"); _plt.xlabel(self.param); _plt.ylabel("Loss"); _plt.show()
    def __repr__(self):
        return f"{super().__repr__()}, use...\n" +\
                "- `.run()` to start scanning for suitable params. This will automatically plots it\n" +\
                "- `.plot()` to plot"
Callbacks.withParamFinder = lambda self, param="lr": self.append(ParamFinder(param))
Callbacks.withParamFinder.__doc__ = ParamFinder.__doc__
Callback.cls.ParamFinder = ParamFinder

In [3]:
!../../export.py _callbacks/paramFinder

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> build/lib/k1lib
copying k1lib/__init__.py ->